## The MDP Model

In [284]:
%matplotlib inline
import scipy.interpolate as intrp
from scipy.stats import norm
from collections import OrderedDict
from collections import Counter
from collections import defaultdict
import matplotlib.pylab as pl
import matplotlib as mpl
import random
import numpy as np
import pandas as pd
import math
from scipy.stats import entropy

#set a font for plots
pl.rcParams.update({'font.size': 18})


#set a seed
random.seed(0)
np.random.seed(0)


In [285]:
'''defining the transition matrices, different transition matrices for positive and negative
outcomes, as well as different matrices for different income classes'''
transitionMatrix2_low_inc = [[1/9,8/27,8/27,8/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]
transitionMatrix_low_inc = [[(5)/9,(4)/27,(4)/27,(4)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]


transitionMatrix2_mid_inc = [[(4.5)/9,(4.5)/27,(4.5)/27,(4.5)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]
transitionMatrix_mid_inc = [[(4.5)/9,(4.5)/27,(4.5)/27,(4.5)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]


transitionMatrix2_high_inc = [[(5)/9,(4)/27,(4)/27,(4)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]
transitionMatrix_high_inc = [[1/9,8/27,8/27,8/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]


'''state names, SS: stay in the same state, SP1/2/3:Positive state 1/2/3, SN1/2/3: Negatiave state 1/2/3'''
transitionName = [["SS","SP1","SP2","SP3"]]
transitionName2 = [["SS2","SN1","SN2","SN3"]]


In [286]:

random.seed(0)
np.random.seed(0)   



'''interpolating the income values from the data: https://dqydj.com/average-median-top-household-income-percentiles/'''        
inc_perc_2019 =  [0, 610.12, 4067.48, 7118.1, 9151.84, 10168.71, 11185.58, 12308.21, 13592.52, 14849.37, 15912, 17143.43, 18303.68, 19341.91, 20337.42, 21386.83, 22585.73, 24021.55, 25197.05, 25986.14, 27292.82, 28472.39, 29710.94, 30510.2, 31727.4, 32844.94, 34237.04, 35590.49, 36511.78, 37624.23, 38702.12, 40381.99, 41062.27, 42664.86, 43766.13, 45351.44, 46320.52, 47792.94, 49114.88, 50843.56, 51120.15, 52674.94, 53953.15, 55816.06, 56944.79, 58404, 59995.4, 61027.52, 62557.91, 64093.39, 65953.25, 67225.35, 69039.45, 70736.61, 71992.44, 73744.51, 75899.26, 77109.34, 78874.63, 80867.7, 82366.56, 84420.64, 86474.72, 88471.86, 90961.16, 92536.29, 94841.54, 97184.41, 99805.9, 101891.51, 104403.18, 106873.16, 109407.2, 112059.2, 114916.61, 118074, 121705.24, 124363.34, 128125.77, 132193.25, 135856.02, 140328.22, 144539.08, 149581.75, 153673.64, 159080.34, 165244.62, 171851.23, 178813.74, 187307.67, 197314.7, 207041.07, 219817.04, 235075.19, 252493.18, 276588.96, 310655.16, 365232.67, 483131.76]
prctng = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.6900000000000001, 0.7000000000000001, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
interp_= intrp.interp1d(prctng,inc_perc_2019,fill_value="extrapolate")
values_= interp_(np.random.random(size=10000))     


        
'''Obtaining monthly income '''      
incomes = values_ /(12.0)





'''assigning savings to different income deciles based on the data:  https://www.federalreserve.gov/econres/scf/dataviz/scf/table/#series:Net_Worth;demographic:inccat;population:all;units:median'''
savings  = [None] * 10000
for y in range (0,10000):
    if incomes[y] < np.percentile(incomes, 20):
        savings[y] =  (9.30*1000) 
        
    if np.percentile(incomes, 20)<=incomes[y] and incomes[y]< np.percentile(incomes, 40):
        savings[y] = (43.80*1000)
  
    if np.percentile(incomes, 40) <=incomes[y] and incomes[y]< np.percentile(incomes, 60):
        savings[y] = (95.62*1000)
        
    if np.percentile(incomes, 60) <=incomes[y] and incomes[y]< np.percentile(incomes, 80) :
        savings[y] = (205.20*1000)
        
    if np.percentile(incomes, 80)  <=incomes[y] and incomes[y]< np.percentile(incomes, 90) :
        savings[y] = (387.85*1000)
        
    if incomes[y]>=np.percentile(incomes, 90):
        savings[y] =  (1589.30*1000)

In [287]:
'''various classifiers and income/savings percentiles'''
clf_threshold_Q1 = int(np.quantile(incomes,0.90))
clf_threshold_Q2 = int(np.quantile(incomes,0.80))
clf_threshold_Q3 = int(np.quantile(incomes,0.70))
clf_threshold_Q4 = int(np.quantile(incomes,0.60))
clf_threshold_Q5 = int(np.quantile(incomes,0.50))


inc_5 = (np.percentile(incomes, 5))
inc_15 = (np.percentile(incomes, 15))
inc_10 = (np.percentile(incomes, 10))
inc_25 = (np.percentile(incomes, 25))
inc_20 = (np.percentile(incomes, 20))
inc_35 = (np.percentile(incomes, 35))
inc_30= (np.percentile(incomes, 30))
inc_40 =(np.percentile(incomes, 40))
inc_45 = (np.percentile(incomes, 45))
inc_50 =(np.percentile(incomes, 50))
inc_55 = (np.percentile(incomes, 55))
inc_60 = (np.percentile(incomes, 60))
inc_65 = (np.percentile(incomes, 65))
inc_70 =(np.percentile(incomes, 70))
inc_75 = (np.percentile(incomes, 75))
inc_80 =(np.percentile(incomes, 80))
inc_85 = (np.percentile(incomes, 85))
inc_90= (np.percentile(incomes, 90))
inc_95 = (np.percentile(incomes, 95))
inc_99 =(np.percentile(incomes, 99))


'''using interpolation based on data just to get the savings deciles: https://dqydj.com/average-median-top-net-worth-percentiles/'''
saving_val = [-94516.52, -54867.26, -35734.82, 
              -25936.24, -18386.58, -11926.06, -7469.49, -3835.64, 
              -1832.38, -466.58, 3.43, 178.2, 502.44, 860.92, 1577.19, 
              2592.8, 3452.04, 4480.43, 5378.37, 6368.35, 7363.47, 8511.79, 9689.16, 
              10777.34, 12430.11, 14197.79, 16060.99, 18106.62, 20340.88, 23695.62, 27250.07, 
              30994.35, 36382.96, 40616.62, 45081.82, 50409.19, 55408.58, 59543.95, 63103.9, 67469.07, 
              71802.48, 75801.39, 80406.68, 84408.2, 89548.35, 94983.2, 100589.66, 106335.51, 114194.83, 
              121411.37, 127446.43, 134207.73, 141621.65, 150028.88, 158430.46, 167469.68, 174943.76, 182100.06, 
              191929.7, 201311.36, 211096.7, 219405.74, 228563.2, 238313.46, 249137.43, 260147.01, 271875.08, 
              288498.14, 301999.85, 314920.61, 328617.37, 349362.49, 365919.18, 382911.64, 403283.56, 428623.03, 
              455610.73, 485176.88, 523925.5, 558189.68, 591350.95, 637050.12, 681782.41, 737122.98, 795218.85, 854908.75, 
              928665.81, 991188.75, 1085969.92, 1219126.46, 1355268.26, 1541905.98, 1767510.16, 2080569.86, 2584130.26, 3294388.49, 4640603.15, 6557022.79, 11099166.07]

savings_prctng= [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.6900000000000001, 0.7000000000000001, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
interp_s=intrp.interp1d(savings_prctng,saving_val,fill_value="extrapolate")
savings_s=interp_s(np.random.random(size=10000))

savings_10 = (np.percentile(savings_s, 10))
savings_20 = (np.percentile(savings_s, 20))
savings_30= (np.percentile(savings_s, 30))
savings_40 =(np.percentile(savings_s, 40))
savings_50 =(np.percentile(savings_s, 50))
savings_60 = (np.percentile(savings_s, 60))
savings_70 =(np.percentile(savings_s, 70))
savings_80 =(np.percentile(savings_s, 80))
savings_90= (np.percentile(savings_s, 90))
savings_99 =(np.percentile(savings_s, 99))

### Health Index

In [288]:
inc_10_sum=[]
inc_20_sum=[]
inc_30_sum=[]
inc_40_sum=[]
inc_50_sum=[]
inc_60_sum=[]
inc_70_sum=[]
inc_80_sum=[]
inc_90_sum=[]
inc_99_sum=[]
inc_99_more = []

for k in incomes:
        if k < inc_10:
            inc_10_sum.append(k)
            
        if k >= inc_10 and k < inc_20:
            inc_20_sum.append(k)
            
        if  k >= inc_20 and k < inc_30:
            inc_30_sum.append(k)
            
        if k >= inc_30 and k < inc_40:
            inc_40_sum.append(k)
            
        if k >= inc_40 and k < inc_50:
            inc_50_sum.append(k)
            
        if k >= inc_50 and k < inc_60:
            inc_60_sum.append(k)
            
        if k >= inc_60 and k < inc_70:
            inc_70_sum.append(k)
            
        if k >= inc_70 and k < inc_80:
            inc_80_sum.append(k)
            
        if k >= inc_80 and k < inc_90:
            inc_90_sum.append(k)
            
        if k >= inc_90 and k < inc_99 :
            inc_99_sum.append(k)
            
        if k >= inc_99:
            inc_99_more.append(k)

inc_10_avg = np.mean(inc_10_sum)
inc_20_avg = np.mean(inc_20_sum)
inc_30_avg = np.mean(inc_30_sum)
inc_40_avg = np.mean(inc_40_sum)
inc_50_avg = np.mean(inc_50_sum)
inc_60_avg = np.mean(inc_60_sum)
inc_70_avg = np.mean(inc_70_sum)
inc_80_avg = np.mean(inc_80_sum)
inc_90_avg = np.mean(inc_90_sum)
inc_99_avg = np.mean(inc_99_sum)
inc_99_avg_more = np.mean(inc_99_more)



def inc_mean(wage):
    avg_y  = 0  
    
    if wage < inc_10:
            avg_y = inc_10_avg
    if wage >= inc_10 and wage < inc_20:
            avg_y= inc_20_avg
    if  wage >= inc_20 and wage < inc_30:
            avg_y= inc_30_avg
    if wage >= inc_30 and wage < inc_40:
            avg_y= inc_40_avg
    if wage >= inc_40 and wage < inc_50:
            avg_y= inc_50_avg
    if wage >= inc_50 and wage < inc_60:
            avg_y= inc_60_avg
    if wage >= inc_60 and wage < inc_70:
            avg_y= inc_70_avg
    if wage >= inc_70 and wage < inc_80:
            avg_y= inc_80_avg
    if wage >= inc_80 and wage < inc_90:
            avg_y= inc_90_avg
    if wage >= inc_90 and wage < inc_99 :
            avg_y= inc_99_avg
    if wage >= inc_99 :
            avg_y= inc_99_avg_more
 
    return avg_y


def final_health_index(income):
    
    ''' calculating the mean health index using the data, each health index is worth 10 points: https://data.census.gov/mdat/#/search?ds=CPSASEC2019&cv=HPCTCUT(0,1,2,4,6,8,10,12,14,16,18,20)&rv=HEA&wt=HSUP_WGT '''    
    h_bar = sum([54325409*50, 53753316*40, 42539580*30, 15096670*20,5565496*10])/sum([54325409,53753316,42539580,15096670,5565496])
    y_is  = income
    y_bar = inc_mean(income)
    
    '''the coefficients are assigned such that we get enough variability to see the difference in precarity indexes. The coefficients
    should maintain the positive first derivative and negative second derivative and as long as they maintain that only the std
    of the distrbutions will change with mean remaining the same'''

    h_is = h_bar  +  1*(y_is - y_bar) - 1e-20*(math.pow((y_is - y_bar),2))

    return h_is 
    
health = []
for k in (incomes):
    health.append(final_health_index(k))
len(health)    


health_10 = (np.percentile(health, 10))
health_20 = (np.percentile(health, 20))
health_30= (np.percentile(health, 30))
health_40 =(np.percentile(health, 40))
health_50 =(np.percentile(health, 50))
health_60 = (np.percentile(health, 60))
health_70 =(np.percentile(health, 70))
health_80 =(np.percentile(health, 80))
health_90= (np.percentile(health, 90))
health_99 =(np.percentile(health, 99))




### Basic Expenditures

Basic expenditures based on the data: https://www.bls.gov/cex/tables/calendar-year/mean-item-share-average-standard-error/cu-income-deciles-before-taxes-2019.pdf

In [289]:
def basic_expenditure(income):
    basic_needs = 0
    if income <= inc_10:
        basic_needs = 25856/12
    if income > inc_10 and income <= inc_20:
        basic_needs = 31499/12
    if income > inc_20 and income<= inc_30:
        basic_needs = 37131/12
    if income > inc_30 and income <= inc_40:
        basic_needs = 43822/12
    if income > inc_40 and income <=inc_50:
        basic_needs = 49367/12
    if income > inc_50 and income <=inc_60:
        basic_needs = 56720/12
    if income > inc_60 and income <=inc_70:
        basic_needs = 66435/12
    if income > inc_70 and income <=inc_80:
        basic_needs = 75945/12
    if income >inc_80 and income <=inc_90:
        basic_needs = 96913/12
    if income >inc_90:
        basic_needs = 145967/12
    return basic_needs

Setting the income classes threshold based on the data: https://www.pewresearch.org/fact-tank/2020/07/23/are-you-in-the-american-middle-class/

In [290]:
low_inc_thrs = np.percentile(incomes, 29)
mid_inc_thrs = np.percentile(incomes, 29 + 52)

### The MDP function (Household tracker)

In [291]:
### the score unit of health index ###
h_u = 10
def individual_tracker(indiv, indiv_health, indiv_savings, indiv_ID, rounds, clf_threshold):

  

    i = 0
    
    indiv_income = indiv

    while i != (rounds):                
                if indiv >= clf_threshold and indiv < low_inc_thrs:
                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_low_inc[0])
                  
                    if change == "SS":

                        (a).append("SS")
                        indiv_income = indiv
                        ''' indiv_savings are actually the networth values'''
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        '''this if loop ensures that the income stays zero and the households burn through their savings at this point'''
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)
                        pass
                

                    elif change == "SP1":
                        a.append("SP1")
                        '''the 10% of income addition/substraction for positive/negative outcomes '''              
                        indiv_income = indiv +  0.1*(indiv_income)
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income =indiv + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        
                        
                    else:
                 
                        a.append("SP3")
                        indiv_income = indiv + 0.1*(indiv_income)  
                        '''500 USD consuing more which is the approximate monthly difference between income deciles (especially lower deciles) in the data: https://www.bls.gov/cex/tables/calendar-year/mean-item-share-average-standard-error/cu-income-deciles-before-taxes-2019.pdf'''
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        
                        
                   
                if indiv >= clf_threshold and mid_inc_thrs> indiv and indiv >= low_inc_thrs:
                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_mid_inc[0])
                    if change == "SS":

                        a.append("SS")
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        pass
                    

                    elif change == "SP1":
                        a.append("SP1")
                        indiv_income =indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                    else:
                        a.append("SP3")
                        indiv_income = indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        
                        
                    

                if indiv >= clf_threshold and indiv >= mid_inc_thrs:

                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_high_inc[0])
                    if change == "SS":

                        a.append("SS")
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        pass
                    

                    elif change == "SP1":
                        a.append("SP1")
                        indiv_income = indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                    else:
                       
                        a.append("SP3")
                        indiv_income = indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        
                        
                       
                if indiv < clf_threshold and indiv < low_inc_thrs:

                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_low_inc[0])
                    
    
                    if change == "SS2":
       
                        a.append("SS")
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)
                        pass
                    
                    
                    elif change == "SN1":

                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        

                    elif change == "SN2":
                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u
                        
                    else:
                        
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)  
                        '''opting out of health insurance plan to cover for liabilitites: health index decrease and netwroth increase by
                        500 USD: the average national cost for health insurance is $456 for an individual (https://www.ehealthinsurance.com/resources/individual-and-family/how-much-does-individual-health-insurance-cost)'''
                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) -(2*h_u)
                        

                if indiv < clf_threshold and  mid_inc_thrs>indiv and indiv>=low_inc_thrs:
                    
                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_mid_inc[0])
                    if change == "SS2":
                        a.append("SS")
                        indiv_income = indiv 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)
                        pass
                    
                    elif change == "SN1":

                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        

                    elif change == "SN2":
                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u

                    else:
                       
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - (2*h_u)
                        
                        
                if indiv < clf_threshold and indiv >=mid_inc_thrs:
                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_high_inc[0])
                    if change == "SS2":
                        a.append("SS")
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)
                        pass
                    
                    
                    
                    elif change == "SN1":

                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)
                        

                    elif change == "SN2":

                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u
                        

                    else:
               
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - (2*h_u)

                indiv = indiv_income

                i += 1
                indiv_health_status.append(indiv_health)
                indiv_savings_status.append(indiv_savings)
                indiv_inc_status.append(indiv_income)
    return indiv



In [292]:
'''calling the MDP Household Tracker function and storing the results'''
g=[]
a = [] 

path_index = []

indiv_health_status = []
indiv_savings_status = []
indiv_inc_status = []
chunks =[]
paths = []
all_inc = []
all_health = []
all_savings = []
rnd = num_rnds = 10


for i in range(0, 10000):
    g.append((individual_tracker(incomes[i], health[i], savings[i], i, rnd, np.percentile(incomes, 50))))

chunks = [a[x:x+rnd] for x in range(0, len(a), rnd)]
paths = [path_index[x:x+rnd] for x in range(0, len(path_index), rnd)]
all_inc = [indiv_inc_status[x:x+rnd] for x in range(0, len(indiv_inc_status), rnd)]
all_health = [indiv_health_status[x:x+rnd] for x in range(0, len(indiv_health_status), rnd)]
all_savings = [indiv_savings_status[x:x+rnd] for x in range(0, len(indiv_savings_status), rnd)]



In [293]:
'''income, savings, and health precarity state percentiles'''

income_class  = []
def income_precarity_states (indiv_inc_status):
    for idx in range(0, len(indiv_inc_status)):
            if indiv_inc_status[idx]<inc_10:
                income_class.append(0)
            if indiv_inc_status[idx]>=inc_10 and indiv_inc_status[idx]<inc_20:
                income_class.append(1)
            if indiv_inc_status[idx]>=inc_20 and indiv_inc_status[idx]<inc_30:
                income_class.append(2)
            if indiv_inc_status[idx]>=inc_30 and indiv_inc_status[idx]<inc_40:
                income_class.append(3)
            if indiv_inc_status[idx]>=inc_40 and indiv_inc_status[idx]<inc_50:
                income_class.append(4)
            if indiv_inc_status[idx]>=inc_50 and indiv_inc_status[idx]<inc_60:
                income_class.append(5)
            if indiv_inc_status[idx]>=inc_60 and indiv_inc_status[idx]<inc_70:
                income_class.append(6)
            if indiv_inc_status[idx]>=inc_70 and indiv_inc_status[idx]<inc_80:
                income_class.append(7)
            if indiv_inc_status[idx]>=inc_80 and indiv_inc_status[idx]<inc_90:
                income_class.append(8)
            if indiv_inc_status[idx]>=inc_90 and indiv_inc_status[idx]<inc_99:
                income_class.append(9)
            if indiv_inc_status[idx]>=inc_99:
                income_class.append(10)
    all_inc_class = [income_class[x:x+rnd] for x in range(0, len(income_class), rnd)] 

    temp = 0
    for idx in range (0, len(incomes)):
            if incomes[idx]<inc_10:
                temp=(0)
            if incomes[idx]>=inc_10 and incomes[idx]<inc_20:
                temp=(1)
            if incomes[idx]>=inc_20 and incomes[idx]<inc_30:
                temp=(2)
            if incomes[idx]>=inc_30 and incomes[idx]<inc_40:
                temp=(3)
            if incomes[idx]>=inc_40 and incomes[idx]<inc_50:
                temp=(4)
            if incomes[idx]>=inc_50 and incomes[idx]<inc_60:
                temp=(5)
            if incomes[idx]>=inc_60 and incomes[idx]<inc_70:
                temp=(6)
            if incomes[idx]>=inc_70 and incomes[idx]<inc_80:
                temp=(7)
            if incomes[idx]>=inc_80 and incomes[idx]<inc_90:
                temp=(8)
            if incomes[idx]>=inc_90 and incomes[idx]<inc_99:
                temp=(9)
            if incomes[idx]>=inc_99:
                temp=(10)
            all_inc_class[idx].insert(0, temp)
            
    return all_inc_class 



savings_class = []
def savings_precarity_states (indiv_savings_status):
    for idx in range(0, len(indiv_savings_status)):
            if indiv_savings_status[idx] <savings_10:
                savings_class.append(0)
            if indiv_savings_status[idx]>=savings_10 and indiv_savings_status[idx]<savings_20:
                savings_class.append(1)
            if indiv_savings_status[idx]>=savings_20 and indiv_savings_status[idx]<savings_30:
                savings_class.append(2)
            if indiv_savings_status[idx]>=savings_30 and indiv_savings_status[idx]<savings_40:
                savings_class.append(3)
            if indiv_savings_status[idx]>=savings_40 and indiv_savings_status[idx]<savings_50:
                savings_class.append(4)
            if indiv_savings_status[idx]>=savings_50 and indiv_savings_status[idx]<savings_60:
                savings_class.append(5)
            if indiv_savings_status[idx]>=savings_60 and indiv_savings_status[idx]<savings_70:
                savings_class.append(6)
            if indiv_savings_status[idx]>=savings_70 and indiv_savings_status[idx]<savings_80:
                savings_class.append(7)
            if indiv_savings_status[idx]>=savings_80 and indiv_savings_status[idx]<savings_90:
                savings_class.append(8)
            if indiv_savings_status[idx]>=savings_90 and indiv_savings_status[idx]<savings_99:
                savings_class.append(9)
            if indiv_savings_status[idx]>=savings_99:
                savings_class.append(10)
    all_savings_class = [savings_class[x:x+rnd] for x in range(0, len(savings_class), rnd)]


    for idx in range(0, len(savings)):

            if savings[idx] <savings_10:
                temp = 0
            if savings[idx]>=-savings_10 and savings [idx]<savings_20:
                temp = (1)
            if savings [idx]>=savings_20 and savings [idx]<savings_30:
                temp = (2)
            if savings [idx]>=savings_30 and savings [idx]<savings_40:
                temp = (3)
            if savings [idx]>=savings_40 and savings [idx]<savings_50:
                temp = (4)
            if savings [idx]>=savings_50 and savings [idx]<savings_60:
                temp = (5)
            if savings [idx]>=savings_60 and savings [idx]<savings_70:
                temp = (6)
            if savings [idx]>=savings_70 and savings [idx]<savings_80:
                temp = (7)
            if savings [idx]>=savings_80 and savings [idx]<savings_90:
                temp = (8)
            if savings [idx]>= savings_90 and savings [idx]< savings_99:
                temp = (9)
            if savings [idx]>=savings_99:
                temp = (10)
            all_savings_class[idx].insert(0, temp)
    return all_savings_class
            
health_class = []
def health_precarity_states (indiv_health_status):
    for idx in range(0, len(indiv_health_status)):
            if indiv_health_status[idx] <health_10:
                health_class.append(0)
            if indiv_health_status[idx]>=health_10 and indiv_health_status[idx]<health_20:
                health_class.append(1)
            if indiv_health_status[idx]>=health_20 and indiv_health_status[idx]<health_30:
                health_class.append(2)
            if indiv_health_status[idx]>=health_30 and indiv_health_status[idx]<health_40:
                health_class.append(3)
            if indiv_health_status[idx]>=health_40 and indiv_health_status[idx]<health_50:
                health_class.append(4)
            if indiv_health_status[idx]>=health_50 and indiv_health_status[idx]<health_60:
                health_class.append(5)
            if indiv_health_status[idx]>=health_60 and indiv_health_status[idx]<health_70:
                health_class.append(6)
            if indiv_health_status[idx]>=health_70 and indiv_health_status[idx]<health_80:
                health_class.append(7)
            if indiv_health_status[idx]>=health_80 and indiv_health_status[idx]<health_90:
                health_class.append(8)
            if indiv_health_status[idx]>=health_90 and indiv_health_status[idx]<health_99:
                health_class.append(9)
            if indiv_health_status[idx]>=health_99:
                health_class.append(10)
    all_health_class = [health_class[x:x+rnd] for x in range(0, len(health_class), rnd)]


    for idx in range(0, len(health)):
            if health[idx] <health_10:
                temp = 0
            if health[idx]>=-health_10 and health [idx]<health_20:
                temp = (1)
            if health [idx]>=health_20 and health [idx]<health_30:
                temp = (2)
            if health [idx]>=health_30 and health [idx]<health_40:
                temp = (3)
            if health [idx]>=health_40 and health [idx]<health_50:
                temp = (4)
            if health [idx]>=health_50 and health [idx]<health_60:
                temp = (5)
            if health [idx]>=health_60 and health [idx]<health_70:
                temp = (6)
            if health [idx]>=health_70 and health [idx]<health_80:
                temp = (7)
            if health [idx]>=health_80 and health [idx]<health_90:
                temp = (8)
            if health [idx]>= health_90 and health [idx]< health_99:
                temp = (9)
            if health [idx]>=health_99:
                temp = (10)

            all_health_class[idx].insert(0, temp)
    return all_health_class
            

            
all_inc_class = income_precarity_states (indiv_inc_status)            
all_savings_class = savings_precarity_states (indiv_savings_status)  
all_health_class = health_precarity_states (indiv_health_status)

In [294]:
'''storing the values in a parameter'''
all_inc_orig_ = all_inc
all_health_orig_ = all_health
all_savings_orig_ = all_savings 
all_inc_class_orig_ = all_inc_class

In [295]:
'''Getting a household's status'''

def check_ind_stats (j):
    print("transition of all statuses after several interations")
    print ("income transitions", all_inc[j])
    print ("income class sequence", (all_inc_class[j]))
    print ("savings class sequence", (all_savings_class[j]))
    print ("health class sequence", (all_health_class[j]))
    print ("savings transitions",all_savings[j])
    print ("health transitions", all_health[j])
    print ("transitions", (chunks[j]))
    print ("initial income:", incomes[j])
    print ("initial savings:", savings[j])
    print ("final income:", all_inc[j][rnd-1])
    print ("final savings:", all_savings[j][rnd-1])
    
    
check_ind_stats (778)

transition of all statuses after several interations
income transitions [10323.110982219338, 10323.110982219338, 10323.110982219338, 11355.42208044127, 12490.964288485398, 13740.060717333938, 15114.066789067332, 16625.473467974065, 18288.020814771473, 20116.82289624862]
income class sequence [7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9]
savings class sequence [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
health class sequence [9, 9, 9, 9, 0, 2, 9, 9, 0, 0, 0]
savings transitions [209194.36098221934, 213188.72196443868, 217183.08294665802, 220462.42169376594, 224777.302648918, 229941.28003291858, 236979.26348865256, 240940.82028996, 247064.9244380648, 254917.83066764675]
health transitions [763.0075242235372, 763.0075242235372, 763.0075242235372, -1553.8039731909764, -408.2617651468489, 830.8346637016904, 2204.8407354350848, -5189.31702540254, -3526.769678605132, -1687.9675971279837]
transitions ['SS', 'SS', 'SS', 'SP2', 'SP1', 'SP3', 'SP2', 'SP3', 'SP2', 'SP1']
initial income: 10323.110982219338
initial savin

### Precarity Quantifiction

In [296]:

def getEntropy(sequence, base=2):
    value,counts = np.unique(sequence, return_counts=True)
    return entropy(counts, base=base)

def generateWeight_health(sequence,values, states):

    weights = []
    for i in range(1,len(sequence)):
        m = states.index(sequence[i])
        n = states.index(sequence[i-1])
        weights.append(math.exp( (1e-4)*abs(values[m]-values[n])))
    return weights

def generateWeight(sequence,values, states):

    weights = []
    for i in range(1,len(sequence)):
        m = states.index(sequence[i])
        n = states.index(sequence[i-1])
        weights.append(math.exp( (1e-22)*abs(values[m]-values[n])))
    return weights


def generateWeight_savings(sequence,values, states):


    weights = []
    for i in range(1,len(sequence)):
        m = states.index(sequence[i])
        n = states.index(sequence[i-1])
        weights.append(math.exp((1e-22)*abs(values[m] - values[n])))

    return weights

def complexity(sequence, stateSpace):
    na = len(stateSpace)
    ls= len(sequence)
    currentState = sequence[0]
    nts = 0
    for i in range(len(sequence)):
        if(sequence[i]!=currentState):
            nts += 1
            currentState = sequence[i]
    return math.sqrt((getEntropy(sequence)/math.log2(na))*(nts/(ls-1)))

def precarityQuantification(sequence, stateSpace, weights, lambdav, alpha, beta, init_state):

    currentState = sequence[0]
    init_cost = (11) - init_state
    negativeWts = 0
    positiveWts = 0
 
        
    for i in range(1,len(sequence)):
            if (sequence[i]>=currentState):

                positiveWts += weights[i-1]
                
            else:
  
                negativeWts += weights[i-1]
            currentState = sequence[i]

    totalWts = sum(weights)

    
    negativeQs = negativeWts/totalWts
    positiveQs = positiveWts/totalWts

    
    
    Qs = (negativeQs - positiveQs)

    return lambdav*init_cost+(1-lambdav)*math.pow(complexity(sequence, stateSpace),alpha)*math.pow(1+Qs,beta)



In [297]:
'''networth precarity '''
c_sav, b_sav = 10000,num_rnds
index_precarity_savings = [[0 for x in range(b_sav)] for y in range(c_sav)] 

for k in range (0, len(all_savings_class)):
    for i in range(1,(num_rnds+1)):
        weights_savings =  generateWeight_savings(all_savings_class[k][:i+1],
        values = [savings_10,savings_20,savings_30,savings_40,savings_50,savings_60,
                 savings_70,savings_80,savings_90,savings_99,5000000], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        index_precarity_savings[k][i-1] = (precarityQuantification(all_savings_class[k][:i+1], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], weights_savings , 0.2, 1, 1.2, all_savings_class[k][0]))


In [ ]:
'''income precarity'''

precarity = []
c_inc, b_inc = 10000,num_rnds
index_precarity = [[0 for x in range(b_inc)] for y in range(c_inc)] 

for k in range (0, len(all_inc_class)):
    for i in range(1,num_rnds+1):
        weights =  generateWeight(all_inc_class[k][:i+1],
        values = [inc_10, inc_20, inc_30, inc_40, inc_50, inc_60, inc_70, inc_80, inc_90, inc_99, 800000.00/12], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        index_precarity[k][i-1] = (precarityQuantification(all_inc_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights , 0.2, 1, 1.2, all_inc_class[k][0]))
        

In [ ]:
'''health precarity'''

c_health, b_health = 10000,num_rnds
index_precarity_health = [[0 for x in range(b_health)] for y in range(c_health)] 

for k in range (0, len(all_health_class)):
    for i in range(1,num_rnds+1):
        weights_health =  generateWeight(all_health_class[k][:i+1],
        values = [health_10,health_20,health_30,health_40,health_50,health_60,
                 health_70,health_80,health_90,health_99,50], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        index_precarity_health[k][i-1] = (precarityQuantification(all_health_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_health , 0.2, 1, 1.2, all_health_class[k][0]))
      

In [ ]:
'''save the original precairty values for initial rounds'''
index_precarity_orig = index_precarity
index_precarity_savings_orig = index_precarity_savings
index_precarity_health_orig = index_precarity_health

In [ ]:
'''Low income class income precarity'''
low_precarity = []
c, b = 10000, num_rnds
index_precarity_lower = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] <  np.percentile(incomes, 29) :

        for i in range(1,num_rnds+1):
            weights =  generateWeight(all_inc_class[k][:i+1],
            values = [inc_10, inc_20, inc_30, inc_40, inc_50, inc_60, inc_70, inc_80, inc_90, inc_99, 800000.00/12], 
            states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])     
            
            index_precarity_lower[k][i-1] = (precarityQuantification(all_inc_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights , 0.2, 1, 1.2, all_inc_class[k][0]))


count_low = 0 
for k in  (index_precarity_lower):
    if k != [0] * num_rnds:
        low_precarity.append(k)
        count_low = count_low +1
print ('count of low income HH:', count_low)


'''middle income class income precarity'''
mid_precarity = []
index_precarity_middle = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] >=  np.percentile(incomes, 29) and incomes[k] <np.percentile(incomes, 29 + 52):
#          print ("y")
         for i in range(1,num_rnds+1):
            weights =  generateWeight(all_inc_class[k][:i+1],
            values = [inc_10, inc_20, inc_30, inc_40, inc_50, inc_60, inc_70, inc_80, inc_90, inc_99, 800000.00/12], 
            states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])     
            
            index_precarity_middle[k][i-1] = (precarityQuantification(all_inc_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights , 0.2, 1, 1.2, all_inc_class[k][0]))

count_mid = 0 
for k in  (index_precarity_middle):
    if k != [0] * num_rnds:
        mid_precarity.append(k)
        count_mid = count_mid +1
print ('count of middle income HH:', count_mid)



'''high income class income precarity'''
high_precarity = []
index_precarity_higher = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] >= np.percentile(incomes, 29 + 52):
        for i in range(1,num_rnds+1):
            weights =  generateWeight(all_inc_class[k][:i+1],
            values = [inc_10, inc_20, inc_30, inc_40, inc_50, inc_60, inc_70, inc_80, inc_90, inc_99, 800000.00/12], 
            states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])     
            
            index_precarity_higher[k][i-1] = (precarityQuantification(all_inc_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights , 0.2, 1, 1.2, all_inc_class[k][0]))

count_high = 0 
for k in  (index_precarity_higher):
    if k != [0] * num_rnds:
        high_precarity.append(k)
        count_high = count_high +1
print ('count of high income HH:', count_high)



In [ ]:

'''incomes classes' networth precarity'''


low_precarity_savings = []
index_precarity_low_savings = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] <  np.percentile(incomes, 29) :

        for i in range(1,num_rnds+1):

            weights_savings_low =  generateWeight_savings(all_savings_class[k][:i+1],
            values = [savings_10,savings_20,savings_30,savings_40,savings_50,savings_60,
                     savings_70,savings_80,savings_90,savings_99,5000000], states = [0, 1, 2, 3, 4, 5, 6, 7 ,8, 9, 10])
            index_precarity_low_savings[k][i-1] = (precarityQuantification(all_savings_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_savings_low , 0.2, 1, 1.2, all_savings_class[k][0]))


count_low_savings = 0 
for k in  (index_precarity_low_savings):
    if k != [0] * num_rnds:
        low_precarity_savings.append(k)
        count_low_savings = count_low_savings +1


middle_precarity_savings = []
index_precarity_middle_savings = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] >=  np.percentile(incomes, 29) and incomes[k] <np.percentile(incomes, 29 + 52):

        for i in range(1,num_rnds+1):

            weights_savings_middle =  generateWeight_savings(all_savings_class[k][:i+1],
            values = [savings_10,savings_20,savings_30,savings_40,savings_50,savings_60,
                     savings_70,savings_80,savings_90,savings_99,5000000], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
            index_precarity_middle_savings[k][i-1] = (precarityQuantification(all_savings_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_savings_middle , 0.2, 1, 1.2, all_savings_class[k][0]))


count_middle_savings = 0 
for k in  (index_precarity_middle_savings):
    if k != [0] * num_rnds:
        middle_precarity_savings.append(k)
        count_middle_savings = count_middle_savings +1



high_precarity_savings = []
index_precarity_high_savings = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] >= np.percentile(incomes, 29 + 52):

        for i in range(1,num_rnds+1):

            weights_savings_high =  generateWeight_savings(all_savings_class[k][:i+1],
            values = [savings_10,savings_20,savings_30,savings_40,savings_50,savings_60,
                     savings_70,savings_80,savings_90,savings_99,5000000], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
            index_precarity_high_savings[k][i-1] = (precarityQuantification(all_savings_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_savings_high , 0.2, 1, 1.2, all_savings_class[k][0]))


count_high_savings = 0 
for k in  (index_precarity_high_savings):
    if k != [0] * num_rnds:
        high_precarity_savings.append(k)
        count_high_savings = count_high_savings +1




In [ ]:
'''incomes classes' health precarity'''


low_precarity_health = []
index_precarity_low_health = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] <  np.percentile(incomes, 29) :

        for i in range(1,num_rnds+1):

            weights_health_low =  generateWeight_health(all_health_class[k][:i+1],
            values = [health_10,health_20,health_30,health_40,health_50,health_60,
                     health_70,health_80,health_90,health_99,50], states = [0, 1, 2, 3, 4, 5, 6, 7 ,8, 9, 10])
            index_precarity_low_health[k][i-1] = (precarityQuantification(all_health_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_health_low , 0.2, 1, 1.2, all_health_class[k][0]))


count_low_health = 0 
for k in  (index_precarity_low_health):
    if k != [0] * num_rnds:
        low_precarity_health.append(k)
        count_low_health = count_low_health +1


middle_precarity_health = []
index_precarity_middle_health = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] >=  np.percentile(incomes, 29) and incomes[k] <np.percentile(incomes, 29 + 52):

        for i in range(1,num_rnds+1):

            weights_health_middle =  generateWeight_health(all_health_class[k][:i+1],
            values = [health_10,health_20,health_30,health_40,health_50,health_60,
                     health_70,health_80,health_90,health_99,50], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
            index_precarity_middle_health[k][i-1] = (precarityQuantification(all_health_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_health_middle , 0.2, 1, 1.2, all_health_class[k][0]))


count_middle_health = 0 
for k in  (index_precarity_middle_health):
    if k != [0] * num_rnds:
        middle_precarity_health.append(k)
        count_middle_health = count_middle_health +1



high_precarity_health = []
index_precarity_high_health = [[0 for x in range(b)] for y in range(c)] 

for k in range (0, len(all_inc_class)):
    if incomes[k] >= np.percentile(incomes, 29 + 52):

        for i in range(1,num_rnds+1):

            weights_health_high =  generateWeight_health(all_health_class[k][:i+1],
            values = [health_10,health_20,health_30,health_40,health_50,health_60,
                     health_70,health_80,health_90,health_99,50], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
            index_precarity_high_health[k][i-1] = (precarityQuantification(all_health_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_health_high , 0.2, 1, 1.2, all_health_class[k][0]))


count_high_health = 0 
for k in  (index_precarity_high_health):
    if k != [0] * num_rnds:
        high_precarity_health.append(k)
        count_high_health = count_high_health +1


### Results

In [ ]:

bucket_coarse = {'=<0.5': 0, '0.5<<=1' : 0,  '1<<1.8' : 0, '>1.8':0 } 


def column(m, i):
    return [row[i] for row in m]

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u  
                
            if (0.5 < exam[i] <= 1):
                 bucket_coarse['0.5<<=1'] += u
      
            if (1 < exam[i] <=1.8 ):
                 bucket_coarse['1<<1.8'] += u

            if (1.8 < exam[i]):
                bucket_coarse['>1.8'] += u
                
    return bucket_coarse


c = np.arange(8, 19)
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Blues)
cmap.set_array([])


bucket_coarse = {key: 0 for key in bucket_coarse}

y = list((plotter_distribution(column(index_precarity_savings, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

width = 0.05
ind = np.arange(4)

bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_savings, 0),1)).values())
pl.figure(5, figsize=(20, 10))
pl.bar(ind, y, label='Round 1',width=width,align='edge',color='g')
pl.legend()

i = 2
for it in range(1, num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity_savings, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+ str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Net worth')
        pl.figure(5, figsize=(20, 10))

        bucket_coarse = {key: 0 for key in bucket_coarse}
        y = list((plotter_distribution(column(index_precarity_savings, it),1)).values())
        pl.bar(ind+(i-1)*width,  y, label='Round '+str(it+1), width=width,align='edge',color=cmap.to_rgba(c[i]))
        pl.legend()
        print (bucket_coarse.values())

        i+=1



pl.ylabel('Number of Households')
pl.xlabel('Precarity Value - Net worth')
pl.show()



In [ ]:

bucket_coarse = {'>=0.2': 0,'0.2<<=0.5':0, '0.5<<=0.8' : 0, '0.8<<=1.2' : 0, '1.2<<=1.5' : 0, '1.5<<=2.5' : 0, '2.5>':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
 
            if (0.2>= exam [i]):
                bucket_coarse['>=0.2'] += u  
                
            if (0.2 < exam [i] <= 0.5):
                bucket_coarse['0.2<<=0.5'] += u
                
            if (0.5 < exam [i] <= 0.8):
                bucket_coarse['0.5<<=0.8'] += u 
                
            if (0.8 < exam[i] <= 1.2):
                 bucket_coarse['0.8<<=1.2'] += u
                    
            if (1.2< exam[i] <= 1.5):
                 bucket_coarse['1.2<<=1.5'] += u
                    
            if (1.5 < exam[i] <=2.5 ):
                 bucket_coarse['1.5<<=2.5'] += u
                    
            if (2.5< exam[i] ):
                 bucket_coarse['2.5>'] += u
     
                     
    return bucket_coarse


bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

width = 0.05
ind = np.arange(7)

bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity, 0),1)).values())
pl.figure(5, figsize=(20, 10))
pl.bar(ind, y, label='Round 1',width=width,align='edge',color='g')
pl.legend()

i = 2
for it in range(1,num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}

        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+ str(it+1),c=cmap.to_rgba(c[i]),markevery=1)

        pl.xticks(rotation='horizontal')

        pl.legend(loc=2, prop={'size': 14})
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Income')


        pl.figure(5, figsize=(20, 10))
        bucket_coarse = {key: 0 for key in bucket_coarse}
        y = list((plotter_distribution(column(index_precarity, it),1)).values())
        pl.bar(ind+(i-1)*width,  y, label='Round '+str(it+1), width=width,align='edge',color=cmap.to_rgba(c[i]))
        pl.legend()
        print (bucket_coarse.values())
        

        i+=1
        



pl.ylabel('Number of Households')
pl.xlabel('Precarity Value - Income')
pl.show()



In [ ]:

bucket_coarse = {'=<0.5': 0,'0.5<<=1':0,  '1<<=1.5':0 ,'1.5<<=2':0, '2<<=2.5':0, '2.5>':0} 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u     
            if (0.5 < exam [i] <= 1):
                bucket_coarse['0.5<<=1'] += u
            if ( 1 <exam[i] <=1.5 ):
                 bucket_coarse['1<<=1.5'] += u
            if ( 1.5<exam[i]<=2):
                 bucket_coarse['1.5<<=2'] += u
            if ( 2<exam[i] <= 2.5 ):
                 bucket_coarse['2<<=2.5'] += u
            if ( 2.5<exam[i]  ):
                 bucket_coarse['2.5>'] += u
          
                
    return bucket_coarse


bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_health, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)#, width=-0.4, align='edge')
pl.xticks(rotation='horizontal')
pl.legend()

width = 0.05
ind = np.arange(6)

bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_health, 0),1)).values())
pl.figure(5, figsize=(20, 10))
pl.bar(ind, y, label='Round 1',width=width,align='edge',color='g')
pl.legend()

i = 2
for it in range(1, num_rnds):
     if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity_health, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Health')
        pl.figure(5, figsize=(20, 10))
        bucket_coarse = {key: 0 for key in bucket_coarse}


        y = list((plotter_distribution(column(index_precarity_health, it),1)).values())
        pl.bar(ind+(i-1)*width,  y, label='Round '+str(it+1), width=width,align='edge',color=cmap.to_rgba(c[i]))
        pl.legend()
        print (bucket_coarse.values())
        

        i+=1
        


pl.ylabel('Number of Households')
pl.xlabel('Precarity Value - Health')
pl.show()

In [ ]:
bucket_coarse = {'>=0.2': 0,'0.2<<=1':0, '1<<=2' : 0, '2<<=3' : 0, '3>':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
 
            if (0.8>= exam [i]):
                bucket_coarse['>=0.2'] += u  
                
            if (0.8 < exam [i] <= 1):
                bucket_coarse['0.2<<=1'] += u
                
            if (1 < exam [i] <= 2):
                bucket_coarse['1<<=2'] += u 
                
            if (2 < exam[i] <= 3):
                 bucket_coarse['2<<=3'] += u
     
            if (3< exam[i]):
                 bucket_coarse['3>'] += u  
                
    return bucket_coarse


bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(low_precarity, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1,num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(low_precarity, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        print (bucket_coarse.values())

        i+=1
        



pl.ylabel('Number of Households')
pl.xlabel('Income Precarity - Lower Income Class')
pl.show()

In [ ]:
bucket_coarse = {'=<0.5': 0, '0.5<<=1' : 0,  '1<<1.8' : 0, '>1.8':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u  

            if (0.5 < exam[i] <= 1):
                 bucket_coarse['0.5<<=1'] += u
      
            if (1 < exam[i] <=1.8 ):
                 bucket_coarse['1<<1.8'] += u

            if (1.8 < exam[i]):
                bucket_coarse['>1.8'] += u
                
    return bucket_coarse



bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(low_precarity_savings, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1, num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(low_precarity_savings, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        print (bucket_coarse.values())
#         pl.ylabel('Number of Households')
#         pl.xlabel('Net worth Precarity - Lower Income Class')

        i+=1


pl.ylabel('Number of Households')
pl.xlabel('Net worth Precarity - Lower Income Class')
pl.show()


In [ ]:
bucket_coarse = {'>=0.2': 0,'0.2<<=1':0, '1<<=2' : 0, '2<<=3' : 0, '3>':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
 
            if (0.8>= exam [i]):
                bucket_coarse['>=0.2'] += u  
                
            if (0.8 < exam [i] <= 1):
                bucket_coarse['0.2<<=1'] += u
                
            if (1 < exam [i] <= 2):
                bucket_coarse['1<<=2'] += u 
                
            if (2 < exam[i] <= 3):
                 bucket_coarse['2<<=3'] += u
    
            if (3< exam[i]):
                 bucket_coarse['3>'] += u    
                
    return bucket_coarse


bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(high_precarity, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()


i = 2
for it in range(1,num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):

        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(high_precarity, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        print (bucket_coarse.values())
        i += 1
        
pl.ylabel('Number of Households')
pl.xlabel('Income Precarity - Higher Income Class')
pl.show()

In [ ]:

bucket_coarse = {'>0.2': 0,'0.2<<0.5':0, '0.5<<0.8' : 0, '0.8<<1.2' : 0, '1.2<<1.5' : 0, '1.5<<2' : 0, '2>':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
 
            if (0.2>= exam [i]):
                bucket_coarse['>0.2'] += u  
                
            if (0.2 < exam [i] <= 0.5):
                bucket_coarse['0.2<<0.5'] += u
                
            if (0.5 < exam [i] <= 0.8):
                bucket_coarse['0.5<<0.8'] += u 
                
            if (0.8 < exam[i] <= 1.2):
                 bucket_coarse['0.8<<1.2'] += u
                    
            if (1.2< exam[i] <= 1.5):
                 bucket_coarse['1.2<<1.5'] += u
                    
            if (1.5 < exam[i] <=2 ):
                 bucket_coarse['1.5<<2'] += u
                    
            if (2< exam[i]):
                 bucket_coarse['2>'] += u
     
                
    return bucket_coarse
# c = np.arange(10, 20)
bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(mid_precarity, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()



i = 2
for it in range(1,num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(mid_precarity, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        pl.ylabel('Number of Households')
        pl.xlabel('Income Precarity - Middle Income Class')
        print (bucket_coarse.values())
        i+=1



pl.show()



In [ ]:
bucket_coarse = {'=<0.5': 0, '0.5<<=1' : 0,  '1<<1.8' : 0, '>1.8':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u  

            if (0.5 < exam[i] <= 1):
                 bucket_coarse['0.5<<=1'] += u
      
            if (1 < exam[i] <=1.8 ):
                 bucket_coarse['1<<1.8'] += u
                    
            if (1.8 < exam[i]):
                bucket_coarse['>1.8'] += u
                
    return bucket_coarse



bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(middle_precarity_savings, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()


i = 2
for it in range(1,num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(middle_precarity_savings, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        print (bucket_coarse.values())


        i+=1
    


pl.ylabel('Number of Households')
pl.xlabel('Net worth Precarity - Middle Income Class')
pl.show()




In [ ]:
bucket_coarse = {'=<0.5': 0, '0.5<<=1' : 0,  '1<<1.8' : 0, '>1.8':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u  
                
            if (0.5 < exam[i] <= 1):
                 bucket_coarse['0.5<<=1'] += u
      
            if (1 < exam[i] <=1.8 ):
                 bucket_coarse['1<<1.8'] += u
  
            if (1.8 < exam[i]):
                bucket_coarse['>1.8'] += u
                
    return bucket_coarse



bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(high_precarity_savings, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()


i = 2
for it in range(1,num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(high_precarity_savings, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        print (bucket_coarse.values())

        i+=1
        

pl.ylabel('Number of Households')
pl.xlabel('Net worth Precarity - Higher Income Class')
pl.show()

In [ ]:
bucket_coarse = {'=<0.5': 0,'0.5<<=1':0,  '1<<=1.5':0 ,'1.5<<=2':0, '2<<=2.5':0, '2.5>':0} 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u     
            if (0.5 < exam [i] <= 1):
                bucket_coarse['0.5<<=1'] += u
            if ( 1 <exam[i] <=1.5 ):
                 bucket_coarse['1<<=1.5'] += u
            if ( 1.5<exam[i]<=2):
                 bucket_coarse['1.5<<=2'] += u
            if ( 2<exam[i] <= 2.5 ):
                 bucket_coarse['2<<=2.5'] += u
            if ( 2.5<exam[i]  ):
                 bucket_coarse['2.5>'] += u
          
                
    return bucket_coarse

bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(high_precarity_health, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1,num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(high_precarity_health, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        print (bucket_coarse.values())


        i+=1
    


pl.ylabel('Number of Households')
pl.xlabel('Health Precarity - Higher Income Class')
pl.show()


In [ ]:
bucket_coarse = {'=<0.5': 0,'0.5<<=1':0,  '1<<=1.5':0 ,'1.5<<=2':0, '2<<=2.5':0, '2.5>':0} 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u  
                
            if (0.5 < exam [i] <= 1):
                bucket_coarse['0.5<<=1'] += u
            if ( 1 <exam[i] <=1.5 ):
                 bucket_coarse['1<<=1.5'] += u
            if ( 1.5<exam[i]<=2):
                 bucket_coarse['1.5<<=2'] += u
            if ( 2<exam[i] <= 2.5 ):
                 bucket_coarse['2<<=2.5'] += u
            if ( 2.5<exam[i]  ):
                 bucket_coarse['2.5>'] += u
          
                
    return bucket_coarse

bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(middle_precarity_health, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1, num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(middle_precarity_health, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')
        pl.legend(loc=2, prop={'size': 14})
        print (bucket_coarse.values())

        i+=1
        



pl.ylabel('Number of Households')
pl.xlabel('Health Precarity - Middle Income Class')
pl.show()



In [ ]:
bucket_coarse = {'=<0.5': 0,'0.5<<=1':0,  '1<<=1.5':0 ,'1.5<<=2':0, '2<<=2.5':0, '2.5>':0} 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u  
                
            if (0.5 < exam [i] <= 1):
                bucket_coarse['0.5<<=1'] += u
            if ( 1 <exam[i] <=1.5 ):
                 bucket_coarse['1<<=1.5'] += u
            if ( 1.5<exam[i]<=2):
                 bucket_coarse['1.5<<=2'] += u
            if ( 2<exam[i] <= 2.5 ):
                 bucket_coarse['2<<=2.5'] += u
            if ( 2.5<exam[i]  ):
                 bucket_coarse['2.5>'] += u
          
                
    return bucket_coarse


bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(low_precarity_health, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1, num_rnds):
    if (it == 1 or it == 3 or it ==5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(low_precarity_health, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1),c=cmap.to_rgba(c[i]),markevery=1)
        pl.xticks(rotation='horizontal')

        pl.legend(loc=2, prop={'size': 14})

        i+=1
        
        print (bucket_coarse.values())


pl.ylabel('Number of Households')
pl.xlabel('Health Precarity - Lower Income Class')
pl.show()


### Fixed Stimulus

In [ ]:

def clf_stimulus_change(indiv, indiv_health, indiv_savings, indiv_ID, rounds, clf_threshold, inter_ind):

    activityToday = states[0]

    i = 0
    indiv_income = indiv

    
   
    prob = 1

    
    if activityToday == states[0]: 
        while i != (rounds):
            if (rounds <= inter_ind):
                if indiv >= clf_threshold and indiv < low_inc_thrs:
                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_low_inc[0])

                    if change == "SS":

                        a.append("SS")
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                        pass

                    elif change == "SP1":
                        a.append("SP1")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <= 0  ):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income = indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0

                        indiv_health = final_health_index(indiv_income) 
                        
                    else:
                         
                        a.append("SP3")
                        indiv_income = indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        
                if indiv >= clf_threshold and mid_inc_thrs> indiv >= low_inc_thrs:
                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_mid_inc[0])

                    if change == "SS":

                        a.append("SS")
                     
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0  ):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                        pass

                    elif change == "SP1":
                        a.append("SP1")
                        indiv_income =indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                    else:
                         
                        a.append("SP3")
                        indiv_income = indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        

                if indiv >= clf_threshold and indiv >= mid_inc_thrs:

                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_high_inc[0])
                    if change == "SS":

                        (a).append("SS")
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                        pass

                    elif change == "SP1":
                        a.append("SP1")
                        indiv_income =indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <=0  ):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=0  ):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                    else:
                         
                        a.append("SP3")
                        indiv_income = indiv_income + 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <=0  ):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        
                if indiv < clf_threshold and indiv < low_inc_thrs:
                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_low_inc[0])
                    if change == "SS2":
                        a.append("SS")
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                        pass
                    elif change == "SN1":

                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                    elif change == "SN2":
                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u
                        
                    else:
                        
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)   

                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) -(2*h_u)

                if indiv < clf_threshold and  mid_inc_thrs>indiv>=low_inc_thrs:

                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_mid_inc[0])
                    if change == "SS2":
                        a.append("SS")
                     
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                        pass
                    elif change == "SN1":

                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   

                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                    elif change == "SN2":

    

                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  
                    
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u

                    else:
                      
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)   
               
                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - (2*h_u)
                        
                if indiv < clf_threshold and indiv >=mid_inc_thrs:
                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_high_inc[0])

                    if change == "SS2":
   
                        a.append("SS")
            
                        indiv_income = indiv

                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)


                        pass
                    
                    elif change == "SN1":

                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   

                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)

                    elif change == "SN2":

       

                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  

                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u

                    else:
                       
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - (2*h_u)


 
            if (rounds>inter_ind): 
 
                if indiv >= clf_threshold and indiv < low_inc_thrs:
                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_low_inc[0])
                    if change == "SS":

                        (a).append("SS")
                    
                        indiv_income = indiv

                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                        pass

                    elif change == "SP1":
                        a.append("SP1")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0

                        indiv_health = final_health_index(indiv_income) 
                        
                    else:
                         
                        a.append("SP3")
                        indiv_income = indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <= 0  ):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                       
                if indiv >= clf_threshold and mid_inc_thrs> indiv >= low_inc_thrs:
                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_mid_inc[0])
   
                    if change == "SS":

                        (a).append("SS")
              
                        indiv_income = indiv 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                        pass

                    elif change == "SP1":
                        a.append("SP1")
                        indiv_income =indiv_income + 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                    else:
                        
                        a.append("SP3")
                        indiv_income = indiv_income + 0.1*(indiv_income)  

                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                       

                if indiv >= clf_threshold and indiv >= mid_inc_thrs:

                    change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix_high_inc[0])
                    if change == "SS":

                        (a).append("SS")
                    
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                        pass

                    elif change == "SP1":
                        a.append("SP1")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 100 )
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) + h_u


                    elif change == "SP2":
                        a.append("SP2")
                        indiv_income =indiv_income + 0.1*(indiv_income) 
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <= 0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                    else:
                         
                        a.append("SP3")
                        indiv_income = indiv_income + 0.1*(indiv_income)  
        
 
                        indiv_savings = indiv_savings + indiv_income - (basic_expenditure(indiv_income) + 500)
                        if (indiv_income <= 0 ):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 
                        
                if indiv < clf_threshold and indiv < low_inc_thrs:
                    indiv = indiv + 1500
                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_low_inc[0])

                    if change == "SS2":

                        a.append("SS")
               
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                        pass
                    
                    
                    elif change == "SN1":
                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                    elif change == "SN2":

                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u
                    else:
                        
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)   

                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) -(2*h_u)

                if indiv < clf_threshold and  mid_inc_thrs>indiv>=low_inc_thrs:
                    indiv = indiv + 1500

                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_mid_inc[0])

                    if change == "SS2":
                        a.append("SS")
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 


                        pass
                    elif change == "SN1":

                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) 

                    elif change == "SN2":
                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  

                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u

                    else:
                        
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)   
    
                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - (2*h_u)
                        
                if indiv < clf_threshold and indiv >=mid_inc_thrs:
                    indiv = indiv + 1500

                    change = np.random.choice(transitionName2[0],replace=True,p=transitionMatrix2_high_inc[0])
                    if change == "SS2":
 
                        a.append("SS")
                    
                        indiv_income = indiv
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)

                        pass
                    
                    elif change == "SN1":
   
                        a.append("SN1")
                        indiv_income =  indiv_income - 0.1*(indiv_income)   
                     
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income)

                    elif change == "SN2":

                        a.append("SN2")
                        indiv_income = indiv_income - 0.1*(indiv_income)  
    
                        indiv_savings = indiv_savings + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - h_u

                    else:
                     
                        a.append("SN3")
                        indiv_income = indiv_income - 0.1*(indiv_income)   
                        indiv_savings = indiv_savings + 500 + indiv_income - basic_expenditure(indiv_income)
                        if (indiv_income <=  0):
                            indiv_income = 0
                        indiv_health = final_health_index(indiv_income) - (2*h_u)


                i += 1
                indiv_health_status.append(indiv_health)
                indiv_savings_status.append(indiv_savings)
                indiv_inc_status.append(indiv_income)
                        
    return indiv


In [ ]:

g=[]
a = [] 
path_index = []
chunks=[]
paths = []
all_inc =[]
all_health =[]
all_savings  =[] 
random.seed(0)
np.random.seed(0)
indiv_health_status = []
indiv_savings_status = []
indiv_inc_status = []

states = [[(incomes)],[],[],[],[],[],[]]
for i in range(0, 10000):
    g.append((clf_stimulus_change(states[0][0][i], health[i], savings[i], i, rnd, clf_threshold_Q5, 1)))

chunks = [a[x:x+rnd] for x in range(0, len(a), rnd)]
paths = [path_index[x:x+rnd] for x in range(0, len(path_index), rnd)]
all_inc = [indiv_inc_status[x:x+rnd] for x in range(0, len(indiv_inc_status), rnd)]
all_health = [indiv_health_status[x:x+rnd] for x in range(0, len(indiv_health_status), rnd)]
all_savings = [indiv_savings_status[x:x+rnd] for x in range(0, len(indiv_savings_status), rnd)]

income_class=[]
health_class = []
savings_class = []

all_inc_class = income_precarity_states (indiv_inc_status)            
all_savings_class = savings_precarity_states (indiv_savings_status)  
all_health_class = health_precarity_states (indiv_health_status)


c_, b_ = 10000, num_rnds
index_precarity_health = [[0 for x in range(b_)] for y in range(c_)]
index_precarity = [[0 for x in range(b_)] for y in range(c_)]
index_precarity_savings = [[0 for x in range(b_)] for y in range(c_)]


'''savings precarity'''
for k in range (0, len(all_savings_class)):
    for i in range(1,num_rnds+1):
        weights_savings =  generateWeight_savings(all_savings_class[k][:i+1],
        values = [savings_10,savings_20,savings_30,savings_40,savings_50,savings_60,
                 savings_70,savings_80,savings_90,savings_99,5000000], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        index_precarity_savings[k][i-1] = (precarityQuantification(all_savings_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_savings , 0.2, 1, 1.2, all_savings_class[k][0]))

        
        
'''income precarity'''
for k in range (0, len(all_inc_class)):
    for i in range(1,num_rnds+1):
        weights =  generateWeight(all_inc_class[k][:i+1],
        values = [inc_10, inc_20, inc_30, inc_40, inc_50, inc_60, inc_70, inc_80, inc_90, inc_99, 800000.00/12], states = [0, 1, 2, 3, 4, 5, 6,7,8,9, 10])
        index_precarity[k][i-1] = (precarityQuantification(all_inc_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights , 0.2, 1, 1.2, all_inc_class[k][0]))

     
    
    
'''health precarity'''
for k in range (0, len(all_health_class)):
    for i in range(1,num_rnds+1):
        weights_health =  generateWeight(all_health_class[k][:i+1],
        values = [health_10,health_20,health_30,health_40,health_50,health_60,
                 health_70,health_80,health_90,health_99,50], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        index_precarity_health[k][i-1] = (precarityQuantification(all_health_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_health , 0.2, 2, 1, all_health_class[k][0]))

       

In [ ]:

'''health'''
bucket_coarse = {'=<0.5': 0,'0.5<<=1':0,  '1<<=1.5':0 ,'1.5<<=2':0, '2<<=2.5':0, '2.5>':0} 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u                  
            if (0.5 < exam [i] <= 1):
                bucket_coarse['0.5<<=1'] += u
            if ( 1 <exam[i] <=1.5 ):
                 bucket_coarse['1<<=1.5'] += u
            if ( 1.5<exam[i]<=2):
                 bucket_coarse['1.5<<=2'] += u
            if ( 2<exam[i] <= 2.5 ):
                 bucket_coarse['2<<=2.5'] += u
            if ( 2.5<exam[i]  ):
                 bucket_coarse['2.5>'] += u
          
                
    return bucket_coarse


bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_health_orig, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)#, width=-0.4, align='edge')
pl.xticks(rotation='horizontal')
pl.legend()

width = 0.05
ind = np.arange(6)
bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_health_orig, 0),1)).values())
pl.figure(5, figsize=(20, 10))
pl.bar(ind, y, label='Round 1',width=width,align='edge',color='g')
pl.legend()

width = 0.05
ind = np.arange(6)
i = 2
for it in range(1,num_rnds):
    if (it == 1 or it  == 3 or it == 5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity_health, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1 ),c=cmap.to_rgba(c[i]),markevery=5)
        pl.xticks(rotation='horizontal')

        pl.legend()
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Health')
        pl.legend(loc=2, prop={'size': 14})
        pl.savefig('C:\\Users\\Pegah Nokhiz\\Desktop\\figures\\aies figs/stimulus_health_precarity.png',bbox_inches='tight')


        pl.figure(5, figsize=(20, 10))
        bucket_coarse = {key: 0 for key in bucket_coarse}
        print ("each 10K income bucket people count after each round (bar plot):")

        y = list((plotter_distribution(column(index_precarity_health, it),1)).values())
        pl.bar(ind+(i-1)*width,  y, label='Round '+str(it+1), width=width,align='edge',color=cmap.to_rgba(c[i]))
        pl.legend()
        pl.ylabel('Number of Households')
        pl.xlabel('Health precarity levels')
        i+=1
        
        print (bucket_coarse.values())


pl.show()

In [ ]:
'''net worth'''
bucket_coarse = {'=<0.5': 0, '0.5<<=1' : 0,  '1<<1.8' : 0, '>1.8':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u  
    
            if (0.5 < exam[i] <= 1):
                 bucket_coarse['0.5<<=1'] += u
      
            if (1 < exam[i] <=1.8 ):
                 bucket_coarse['1<<1.8'] += u
 
            if (1.8 < exam[i]):
                bucket_coarse['>1.8'] += u
                
    return bucket_coarse
          
 


bucket_coarse = {key: 0 for key in bucket_coarse}

y = list((plotter_distribution(column(index_precarity_savings_orig, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1,num_rnds):
    if (it == 1 or it  == 3 or it == 5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity_savings, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1 ),c=cmap.to_rgba(c[i]),markevery=5)
        pl.xticks(rotation='horizontal')

        pl.legend()
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Savings')
        print (bucket_coarse.values())
        i+=1

pl.show()


In [ ]:
'''income'''
bucket_coarse = {'>=0.2': 0,'0.2<<=0.5':0, '0.5<<=0.8' : 0, '0.8<<=1.2' : 0, '1.2<<=1.5' : 0, '1.5<<=2.5' : 0, '2.5>':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
 
            if (0.2>= exam [i]):
                bucket_coarse['>=0.2'] += u  
                
            if (0.2 < exam [i] <= 0.5):
                bucket_coarse['0.2<<=0.5'] += u
                
            if (0.5 < exam [i] <= 0.8):
                bucket_coarse['0.5<<=0.8'] += u 
                
            if (0.8 < exam[i] <= 1.2):
                 bucket_coarse['0.8<<=1.2'] += u
                    
            if (1.2< exam[i] <= 1.5):
                 bucket_coarse['1.2<<=1.5'] += u
                    
            if (1.5 < exam[i] <=2.5 ):
                 bucket_coarse['1.5<<=2.5'] += u
                    
            if (2.5< exam[i] ):
                 bucket_coarse['2.5>'] += u

    return bucket_coarse



bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_orig, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1,num_rnds):
    if (it == 1 or it  == 3 or it == 5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1 ),c=cmap.to_rgba(c[i]),markevery=5)
        pl.xticks(rotation='horizontal')

        pl.legend()
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Income')
       
        i+=1
        
        print (bucket_coarse.values())

pl.show()

### Precarity Resistence

In [ ]:
'''new transition matrices'''

transitionMatrix2_low_inc = [[6/9,(3)/27,(3)/27,(3)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]
transitionMatrix_low_inc = [[(5)/9,(4)/27,(4)/27,(4)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]


transitionMatrix2_mid_inc = [[(6)/9,(3)/27,(3)/27,(3)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]
transitionMatrix_mid_inc = [[(4.5)/9,(4.5)/27,(4.5)/27,(4.5)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]


transitionMatrix2_high_inc = [[(5)/9,(4)/27,(4)/27,(4)/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]
transitionMatrix_high_inc = [[1/9,8/27,8/27,8/27], [0,0,0,0], [0,0,0,0], [0,0,0,0]]
g=[]
a = [] 
path_index = []
chunks=[]
paths = []
all_inc =[]
all_health =[]
all_savings  =[] 
random.seed(0)
np.random.seed(0)
indiv_health_status = []
indiv_savings_status = []
indiv_inc_status = []

states = [[(incomes)],[],[],[],[],[],[]]
for i in range(0, 10000):
    g.append((individual_tracker(states[0][0][i], health[i], savings[i], i, rnd, clf_threshold_Q5)))

chunks = [a[x:x+rnd] for x in range(0, len(a),rnd)]
paths = [path_index[x:x+rnd] for x in range(0, len(path_index), rnd)]
all_inc = [indiv_inc_status[x:x+rnd] for x in range(0, len(indiv_inc_status), rnd)]
all_health = [indiv_health_status[x:x+rnd] for x in range(0, len(indiv_health_status),rnd)]
all_savings = [indiv_savings_status[x:x+rnd] for x in range(0, len(indiv_savings_status), rnd)]


income_class=[]
health_class = []
savings_class = []

all_inc_class = income_precarity_states (indiv_inc_status)            
all_savings_class = savings_precarity_states (indiv_savings_status)  
all_health_class = health_precarity_states (indiv_health_status)


c_, b_ = 10000, num_rnds
index_precarity_health = [[0 for x in range(b_)] for y in range(c_)]
index_precarity = [[0 for x in range(b_)] for y in range(c_)]
index_precarity_savings = [[0 for x in range(b_)] for y in range(c_)]


'''savings precarity'''
for k in range (0, len(all_savings_class)):
    for i in range(1,num_rnds+1):
        weights_savings =  generateWeight_savings(all_savings_class[k][:i+1],
        values = [savings_10,savings_20,savings_30,savings_40,savings_50,savings_60,
                 savings_70,savings_80,savings_90,savings_99,5000000], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        index_precarity_savings[k][i-1] = (precarityQuantification(all_savings_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_savings , 0.2, 1, 1.2, all_savings_class[k][0]))

        
        
'''income precarity'''
for k in range (0, len(all_inc_class)):
    for i in range(1,num_rnds+1):

        weights =  generateWeight(all_inc_class[k][:i+1],
        values = [inc_10, inc_20, inc_30, inc_40, inc_50, inc_60, inc_70, inc_80, inc_90, inc_99, 800000.00/12], states = [0, 1, 2, 3, 4, 5, 6,7,8,9, 10])
        index_precarity[k][i-1] = (precarityQuantification(all_inc_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights , 0.2, 1, 1.2, all_inc_class[k][0]))

     
    
    
'''health precarity'''
for k in range (0, len(all_health_class)):
    for i in range(1,num_rnds+1):
        weights_health =  generateWeight(all_health_class[k][:i+1],
        values = [health_10,health_20,health_30,health_40,health_50,health_60,
                 health_70,health_80,health_90,health_99,50], states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        index_precarity_health[k][i-1] = (precarityQuantification(all_health_class[k][:i+1], [0,1,2,3,4,5,6,7,8,9,10], weights_health , 0.2, 2, 1, all_health_class[k][0]))

       

In [ ]:

'''health'''
bucket_coarse = {'=<0.5': 0,'0.5<<=1':0,  '1<<=1.5':0 ,'1.5<<=2':0, '2<<=2.5':0, '2.5>':0} 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u                  
            if (0.5 < exam [i] <= 1):
                bucket_coarse['0.5<<=1'] += u
            if ( 1 <exam[i] <=1.5 ):
                 bucket_coarse['1<<=1.5'] += u
            if ( 1.5<exam[i]<=2):
                 bucket_coarse['1.5<<=2'] += u
            if ( 2<exam[i] <= 2.5 ):
                 bucket_coarse['2<<=2.5'] += u
            if ( 2.5<exam[i]  ):
                 bucket_coarse['2.5>'] += u
          
                
    return bucket_coarse







norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Blues)
cmap.set_array([])
bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_health_orig, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)#, width=-0.4, align='edge')
pl.xticks(rotation='horizontal')
pl.legend()

width = 0.05
ind = np.arange(6)

bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_health_orig, 0),1)).values())
pl.figure(5, figsize=(20, 10))
pl.bar(ind, y, label='Round 1',width=width,align='edge',color='g')
pl.legend()

width = 0.05
ind = np.arange(6)
i = 2
for it in range(1,num_rnds):
    if (it == 1 or it  == 3 or it == 5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity_health, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1 ),c=cmap.to_rgba(c[i]),markevery=5)
        pl.xticks(rotation='horizontal')

        pl.legend()
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Health')
        pl.legend(loc=2, prop={'size': 14})
        pl.savefig('C:\\Users\\Pegah Nokhiz\\Desktop\\figures\\aies figs/markov_health_precarity.png',bbox_inches='tight')


        pl.figure(5, figsize=(20, 10))
        bucket_coarse = {key: 0 for key in bucket_coarse}
        print ("each 10K income bucket people count after each round (bar plot):")

        y = list((plotter_distribution(column(index_precarity_health, it),1)).values())
        pl.bar(ind+(i-1)*width,  y, label='Round '+str(it+1), width=width,align='edge',color=cmap.to_rgba(c[i]))
        pl.legend()
        pl.ylabel('Number of Households')
        pl.xlabel('Health precarity levels')
        i+=1
        
        print (bucket_coarse.values())

pl.show()

In [ ]:
'''net worth'''
bucket_coarse = {'=<0.5': 0, '0.5<<=1' : 0,  '1<<1.8' : 0, '>1.8':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
        
            if (0.5>= exam [i]):
                bucket_coarse['=<0.5'] += u  
    
            if (0.5 < exam[i] <= 1):
                 bucket_coarse['0.5<<=1'] += u
      
            if (1 < exam[i] <=1.8 ):
                 bucket_coarse['1<<1.8'] += u
 
            if (1.8 < exam[i]):
                bucket_coarse['>1.8'] += u
                
    return bucket_coarse
          
 


bucket_coarse = {key: 0 for key in bucket_coarse}

y = list((plotter_distribution(column(index_precarity_savings_orig, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1,num_rnds):
    if (it == 1 or it  == 3 or it == 5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity_savings, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1 ),c=cmap.to_rgba(c[i]),markevery=5)
        pl.xticks(rotation='horizontal')

        pl.legend()
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Net worth')
        print (bucket_coarse.values())
        i+=1

pl.show()


In [ ]:
'''income'''
bucket_coarse = {'>=0.2': 0,'0.2<<=0.5':0, '0.5<<=0.8' : 0, '0.8<<=1.2' : 0, '1.2<<=1.5' : 0, '1.5<<=2.5' : 0, '2.5>':0 } 

def plotter_distribution(exam, u):
    
    for i in range(1,len(exam)):
 
            if (0.2>= exam [i]):
                bucket_coarse['>=0.2'] += u  
                
            if (0.2 < exam [i] <= 0.5):
                bucket_coarse['0.2<<=0.5'] += u
                
            if (0.5 < exam [i] <= 0.8):
                bucket_coarse['0.5<<=0.8'] += u 
                
            if (0.8 < exam[i] <= 1.2):
                 bucket_coarse['0.8<<=1.2'] += u
                    
            if (1.2< exam[i] <= 1.5):
                 bucket_coarse['1.2<<=1.5'] += u
                    
            if (1.5 < exam[i] <=2.5 ):
                 bucket_coarse['1.5<<=2.5'] += u
                    
            if (2.5< exam[i] ):
                 bucket_coarse['2.5>'] += u

    return bucket_coarse



bucket_coarse = {key: 0 for key in bucket_coarse}
y = list((plotter_distribution(column(index_precarity_orig, 0),1)).values())
pl.figure(0, figsize=(20, 10))
pl.plot(list(bucket_coarse.keys()),y, label='Round 1',linewidth=3,c='g',markevery=1000)
pl.xticks(rotation='horizontal')
pl.legend()

i = 2
for it in range(1,num_rnds):
    if (it == 1 or it  == 3 or it == 5 or it == 7 or it == 9):
        bucket_coarse = {key: 0 for key in bucket_coarse}
        pl.figure(0, figsize=(20, 10))
        y = list((plotter_distribution(column(index_precarity, it),1)).values())
        pl.plot(list(bucket_coarse.keys()),y,label='Round '+str(it+1 ),c=cmap.to_rgba(c[i]),markevery=5)
        pl.xticks(rotation='horizontal')

        pl.legend()
        pl.ylabel('Number of Households')
        pl.xlabel('Precarity Value - Income')
       
        i+=1
        
        print (bucket_coarse.values())

pl.show()